In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas
import numpy as np

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [ ]:
spark = SparkSession\
        .builder\
        .master("spark://192.168.2.83:7077") \
        .appName("Aram_testar")\
        .config("spark.dynamicAllocation.enabled", False)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled",False)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

#If you get following error: "WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources" 
#solve by setting dynamicAlloc and Shuffletracking to FALSE

In [ ]:
schema = StructType([\
    StructField("TrackID", StringType(), True),\
    StructField("Genre", StringType(), True)])

million_song_df = spark.read.csv("hdfs://192.168.2.83:9000/user/ubuntu/SongCSVWithTrackID.csv", header=True, inferSchema=True)
million_song_notbyte_df = million_song_df.withColumn("TrackID", regexp_replace(col("TrackID").cast(StringType()), "['b']", ""))

genres_df = spark.read.option("delimiter", "\t").csv("hdfs://192.168.2.83:9000/user/ubuntu/genres.csv", schema=schema)

million_song_notbyte_df.take(1)
# Filter out songs with year=0
filtered_million_song_df = million_song_notbyte_df.filter(col("Year") != 0)

# Join the two DataFrames on the common column (e.g., track_id)
# joined_df = genres_df.join(filtered_million_song_df, genres_df["TrackID"] == million_song_df["TrackID"])
joined_df = genres_df.join(filtered_million_song_df, genres_df["TrackID"] == filtered_million_song_df["TrackID"])


# Group by genre and year, then calculate the percentage of songs in each genre by year
result_df = joined_df.groupBy("Genre", "Year").count()
result_df = result_df.orderBy("Year")
pivot_table = result_df.groupBy("Year").pivot("Genre").agg(count("Genre").alias("count")).fillna(0).orderBy("Year")

pivot_table = result_df


# Show the result or save it as needed
# filtered_million_song_df.take(1)
# joined_df.take(10)
# result_df.take(100)


In [ ]:
# Group by Genre and Year and calculate the count
result_df = joined_df.groupBy("Genre", "Year").count()       

# Pivot the DataFrame to display counts for each genre in each year
pivot_result_df = result_df.groupBy("Year").pivot("Genre").agg(F.first("count"))

# Fill null values with 0
pivot_result_df    =    pivot_result_df.na.fill(0).orderBy("Year")

# Show the result
pivot_result_df.show(100)

In [ ]:
pivot_reslts_Pandas = pivot_result_df.toPandas()
pivot_reslts_Pandas = pivot_reslts_Pandas.set_index("Year")
# pivot_reslts_Pandas = pivot_reslts_Pandas.apply(lambda x: x*100/sum(x), axis=1)

pivot_reslts_Pandas.plot(kind="bar", stacked=True)
# pivot_reslts_Pandas.head()

In [ ]:
# release the cores for another application!
spark_session.stop()

In [ ]:

print(songs_df.take(1))
print(genre_df.take(1))

In [ ]:
genre_df.head() 